In [1]:
import pandas as pd
import numpy as np
from prefect.blocks.system import Secret
from catnip.fla_redshift import FLA_Redshift
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

from datetime import datetime
import statsmodels.api as sm

import polars as pl
import pyarrow

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

PrefectHTTPStatusError: Client error '401 Unauthorized' for url 'https://api.prefect.cloud/api/accounts/fbcf5f47-1244-4f4b-8a37-4ea5344727df/workspaces/c24db6e7-9c24-466a-8d86-94154cc57d8d/block_types/slug/secret/block_documents/name/stellar-redshift-db-name?include_secrets=true'
Response: {'detail': 'Invalid authentication credentials'}
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/401

In [ ]:
q = """
with crossed_2324 as
    (select
        distinct section,
        price_level,
        event_dates.event_datetime
    from
        custom.cth_v_manifest_2324
    cross join
        (select distinct event_datetime from custom.cth_v_ticket_2324) event_dates
    where
        price_level in ('U','V','W','Y')),
temp_2324 as
    (select
        event_datetime,
        section,
        ticket_type,
        datediff('days', min(transaction_date), event_datetime) as days_out
    from
        custom.cth_v_ticket_2324 ticket
    where
        pc_one in ('U','V','W','Y')
    group by
        event_datetime,
        section,
        ticket_type),
crossed_2425 as
    (select
        distinct section,
        price_level,
        event_dates.event_datetime
    from
        custom.cth_v_manifest_2425
    cross join
        (select distinct event_datetime from custom.cth_v_ticket_2425) event_dates
    where
        price_level in ('U','V','W','Y')),
temp_2425 as
    (select
        event_datetime,
        section,
        ticket_type,
        datediff('days', min(transaction_date), event_datetime) as days_out
    from
        custom.cth_v_ticket_2425 ticket
    where
        pc_one in ('U','V','W','Y')
    group by
        event_datetime,
        section,
        ticket_type)
select
    '2023-24' as season,
    crossed_2324.event_datetime,
    crossed_2324.section,
    ticket_type,
    days_out
from
    crossed_2324
left join
    temp_2324 on crossed_2324.section = temp_2324.section
    and crossed_2324.event_datetime = temp_2324.event_datetime
UNION ALL
select
    '2024-25' as season,
    crossed_2425.event_datetime,
    crossed_2425.section,
    ticket_type,
    days_out
from
    crossed_2425
left join
    temp_2425 on crossed_2425.section = temp_2425.section
    and crossed_2425.event_datetime = temp_2425.event_datetime
"""